In [1]:
import re, math
import operator
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import StratifiedShuffleSplit
from random import randint
# import ipynb.fs.defs.PeopleInfo as peopleInfo
from nltk.corpus import stopwords
from nltk import TweetTokenizer
from nltk import PorterStemmer
import ipynb.fs.defs.FilterMethods as filterMethods
import sys, os
sys.path.append('../2_feature')
import ipynb.fs.defs.GetFeatures as getFeatures
global stop_words
stop_words = stopwords.words('english')
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.display import display, Image
plotly.tools.set_credentials_file(username='Adeline', api_key='Z5eltNtBQXqvI05ZFQtz')
from ipywidgets import IntProgress
from IPython.display import display
import datetime
from matplotlib import dates
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.dates as mdates

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [240]:
class Patient(object):
    totalCount = 0
    diagnosedTimeDict = dict()
    idToNameDict = dict()
    with open('../0_dataset/diagnosedTweetsPatch2') as open_file:
        for line in open_file.readlines():
            line = line.strip().split('\t')
            diagnosedTimeDict[line[0]] = datetime.datetime.strptime(line[3], "%Y-%m-%d %H:%M:%S")
            idToNameDict[line[0]] = line[1]
       
    def __init__(self, userId):
        self.userId = userId
        self.name = Patient.idToNameDict[userId]
        self.df0 = pd.read_csv('../0_dataset/DepressionUsersTweets/Patch2_data/'+userId,sep='\t',header=None,usecols=[2,3],names=['Date','Text'],quoting=3,error_bad_lines=False,encoding='utf-8').dropna(axis=0, how='any')
        self.df0 = self.df0.drop_duplicates(subset='Date', keep='first')
        self.df0 = self.df0.set_index('Date')
        self.df0.index = pd.to_datetime(self.df0.index)

        self.diagnosedTime = Patient.diagnosedTimeDict[userId]
        timeDuration1 = self.diagnosedTime - datetime.timedelta(days=90)
        timeDuration2 = self.diagnosedTime - datetime.timedelta(days=180)
        self.df1 = self.df0.loc[str(self.diagnosedTime):str(timeDuration1)]
        self.df2 = self.df0.loc[str(timeDuration1):str(timeDuration2)]
        Patient.totalCount += 1 
        
    def displayCount(self):
        print("Total Patients {0}".format(Patient.totalCount))
    
    def displayTweetsCount(self, category=1):
        if category == 0:
            return len(self.df0)
        elif category == 1:
            return len(self.df1)
        elif category == 2:
            return len(self.df2)
        else:
            print('Please indicate category (inRange or all)')
            return False
        
    def getText(self, category=1):
        if category == 0:
            return self.df0['Text'].values
        if category == 1:
            return self.df1['Text'].values
        elif category == 2:
            return self.df2['Text'].values
        else:
            print('Please indicate category (inRange or all)')
            return False

In [241]:
def replace_by_symbols(txt):
    txt = re.sub(r"https\S+", '', txt)
    txt = re.sub(r"http\S+", '', txt)
    txt = re.sub(r"pic.twitter.com\S+", '', txt)
    txt = re.sub(r"twitter.com/\S+", '', txt)
    txt = re.sub(r"\S+/\S+", '', txt)
    txt = re.sub(r"@\S+", '', txt)
    txt = re.sub(r"#\S+", '', txt)
    txt = re.sub(r"idk", 'i do not know', txt)   # idk: i don't know
    txt = re.sub(r"tbh", 'to be honest', txt)   # tbh: to be honest
    txt = re.sub(r"tho", 'though', txt)   # tho
    txt = re.sub(r"i\'m", 'i am', txt)
    txt = re.sub(r"you\'re", 'you are', txt)
    txt = re.sub(r"he\'s", 'he is', txt)
    txt = re.sub(r"she\'s", 'she is', txt)
    txt = re.sub(r"it\'s", 'it is', txt)
    txt = re.sub(r"we\'re", 'we are', txt)
    txt = re.sub(r"they\'re", 'they are', txt)
    txt = re.sub(r"isn\'t", 'is not', txt)
    txt = re.sub(r"don\'t", 'do not', txt)
    txt = re.sub(r"doesn\'t", 'does not', txt)
    txt = re.sub(r"didn\'t", 'did not', txt)
    txt = re.sub(r"wasn\'t", 'was not', txt)
    txt = re.sub(r"weren\'t", 'were not', txt)
    txt = re.sub(r"haven\'t", 'have not', txt)
    txt = re.sub(r"can\'t", 'can not', txt)
    txt = re.sub(r"couldn\'t", 'could not', txt)
    txt = re.sub(r"wouldn\'t", 'would not', txt)
    txt = re.sub(r"shouldn\'t", 'should not', txt)
    txt = re.sub(r"&amp", '', txt)
    return txt

In [242]:
%%time
patients = dict()
with open('../0_dataset/patch2PatientIDs') as openFile:
    for patient in openFile.readlines():
        patient = patient.strip()
        patients[patient] = Patient(patient)
        # break

CPU times: user 1.6 s, sys: 52 ms, total: 1.66 s
Wall time: 1.66 s


In [5]:
# base_texts = []
group_texts = []

for key in patients.keys():
    group_texts.append(replace_by_symbols('\n'.join(patients[key].getText(1))))

# for key in ordinarys.keys():
#     base_texts.append(replace_by_symbols('\n'.join(ordinarys[key].getText())))

# corpus = base_texts + group_texts

In [6]:
def create_vectorizer(ngrams=1):
    vectorizer = TfidfVectorizer(stop_words="english", token_pattern='(?u)\\b[a-zA-Z]\\w{2,}\\b', ngram_range = (ngrams,ngrams), min_df=1)
    return vectorizer

In [7]:
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    words = []
    scores = []
    for item in sorted_scores:
        words.append(item[0])
        scores.append(item[1])
    return words, scores

# No filtering

In [11]:
vectorizer = create_vectorizer()
tfidf_result = vectorizer.fit_transform(group_texts)
words, scores = display_scores(vectorizer, tfidf_result)

In [18]:
for i in range(50):
    print(words[i],scores[i])

just 15.5710522482
like 14.9442976943
love 11.5760331508
know 8.14639317976
good 6.87465270101
want 6.70333379776
people 6.50405708417
really 6.33557510248
shit 6.06214344154
fuck 6.04191544798
time 5.74418016352
got 5.59225987869
today 5.5766771962
day 5.55936927208
did 5.41035565804
fucking 5.31691932914
going 5.16847757787
think 5.06596968172
need 4.92763499858
new 4.69978960361
gonna 4.65245657381
feel 4.50048453158
lol 4.45591093829
life 4.44754772597
happy 4.38990850656
make 3.94688577686
lmao 3.90652723362
thank 3.77173831567
god 3.72633669
video 3.68074494519
does 3.64053041784
wanna 3.50040100928
follow 3.49097951501
right 3.45836030879
hate 3.43533657602
better 3.20420883639
man 3.05512789452
best 3.01779739779
okay 3.00758546574
look 3.00602663577
thing 2.99519795718
yes 2.98078145333
omg 2.97419351604
friends 2.9119888107
let 2.90469612947
bad 2.90251133436
hope 2.84351825722
work 2.82317582552
actually 2.8015453372
say 2.79317040021


# Filter out users (Case1)

In [20]:
patch2FilterOutCase1 = []
with open('patch2FilterOutCase1') as open_file:
    for line in open_file.readlines():
        line = line.strip()
        patch2FilterOutCase1.append(line)
        try:
            del patients[line]
        except KeyError:
            continue

In [22]:
group_texts = []

for key in patients.keys():
    group_texts.append(replace_by_symbols('\n'.join(patients[key].getText(1))))

In [23]:
vectorizer = create_vectorizer()
tfidf_result = vectorizer.fit_transform(group_texts)
words, scores = display_scores(vectorizer, tfidf_result)

In [24]:
for i in range(50):
    print(words[i],scores[i])

just 12.5693990997
like 12.4306876504
love 9.07527699758
know 6.6528181685
good 5.7216059736
really 5.3027535116
want 5.27326990107
people 5.19184026681
fuck 5.0079775563
shit 4.99594358312
time 4.79412937591
day 4.63950289237
got 4.53774766959
did 4.35256194845
fucking 4.32115320006
today 4.27290431912
think 4.12787881051
going 4.04178678479
gonna 3.88146080441
need 3.83240002137
new 3.78167009083
lol 3.66843185208
happy 3.58478404255
feel 3.53966287542
life 3.51191572816
lmao 3.41314318803
god 3.31839401953
make 3.17128102804
thank 2.97127675015
wanna 2.90731860242
right 2.73336646218
hate 2.72662150802
video 2.60991466044
man 2.55823758137
does 2.55172257944
work 2.49138633262
let 2.49134838183
thing 2.43784951112
better 2.43610531823
look 2.43285245348
best 2.41861538929
omg 2.35168481564
yes 2.33455000143
yeah 2.3145499956
actually 2.29906762588
okay 2.29126109399
say 2.28808025758
hope 2.26109163709
bad 2.23536299484
night 2.20973465369


# Filter (Case2)

In [192]:
def randomSelectRows(userId, rowsPerDay=5):
    
    indices = []
    values = []
    df = patients[userId].df1
    if patients[userId].displayTweetsCount(1) <= 450:
        indices.extend(df.index)
    else:
        gp = df.index.groupby(df.index.day)

        for key in gp.keys():
            try:
                rowIndex = np.random.choice(gp[key], rowsPerDay, replalce=False)
            except:
                rowIndex = np.random.choice(gp[key], rowsPerDay)
            indices.extend(pd.to_datetime(rowIndex))
        indices = list(set(indices))
        i = len(indices)
        while i < 450:
            tmpIndices = pd.to_datetime(np.random.choice(df.index, 1))
#             print('{0}\t{1}'.format(i, tmpIndices))
            if tmpIndices in indices:
#                 print('{0} is already in indcies'.format(tmpIndices))
                continue
            else:
                indices.extend(tmpIndices)
                i += 1
#     print(indices)
    for index in indices:
        if type(df.loc[index]['Text']) != str:
            print(index, df.loc[index]['Text'])
        values.append(df.loc[index]['Text'])
        
    return values

In [243]:
# 每人每天5篇，總共450篇（不滿就全取）
%%time
group_texts = []
print('Total: {0}'.format(len(patients)))
for i, patient in enumerate(patients):
    print('Procesing {0}\t{1}...'.format(i, patient))
    values = randomSelectRows(patient, 5)
    text = replace_by_symbols('\n'.join(values))
    group_texts.append(text)

Procesing 0	2966191458...
Procesing 1	485255179...
Procesing 2	2468363603...
Procesing 3	720699847012192256...
Procesing 4	2246711085...
Procesing 5	4303354228...
Procesing 6	298843567...
Procesing 7	772661900...
Procesing 8	1626298430...
Procesing 9	2478963242...
Procesing 10	4057096000...
Procesing 11	570561813...
Procesing 12	874493282...
Procesing 13	1666467426...
Procesing 14	101959854...
Procesing 15	2851359600...
Procesing 16	562975438...
Procesing 17	3793973969...
Procesing 18	3474584555...
Procesing 19	315708868...
Procesing 20	169073155...
Procesing 21	2966022107...
Procesing 22	3289341230...
Procesing 23	804789932522872833...
Procesing 24	613658426...
Procesing 25	4411064547...
Procesing 26	756912267199995904...
Procesing 27	723170886015012864...
Procesing 28	2843195353...
Procesing 29	564143639...
Procesing 30	28265926...
Procesing 31	1417808906...
Procesing 32	36249905...
Procesing 33	2727420622...
Procesing 34	2793648772...
Procesing 35	2994318326...
Procesing 36	24282835

In [244]:
vectorizer = create_vectorizer()
tfidf_result = vectorizer.fit_transform(group_texts)
words, scores = display_scores(vectorizer, tfidf_result)
for i in range(50):
    print(words[i],scores[i])

just 13.251386251
like 12.6759625785
love 10.1064637363
know 7.05588078773
good 5.71563453578
want 5.68239696985
people 5.62909106717
fuck 5.31558053484
shit 5.20941546475
today 5.13584579144
really 5.12161574733
got 4.97468696209
time 4.88386201809
day 4.77172255352
did 4.70543072846
going 4.52556028881
think 4.45464595889
fucking 4.42159899843
need 4.20096915967
new 4.09025085855
gonna 3.98456210386
lol 3.90347251054
feel 3.80748608081
happy 3.76655322713
life 3.73950027815
video 3.48662167088
make 3.39587988818
follow 3.31755675488
wanna 3.2003063673
does 3.12400829222
hate 3.08508546441
right 3.0575187109
man 2.97186397144
thank 2.92853574343
god 2.90751134198
lmao 2.8268708074
better 2.73961009067
best 2.68009303153
work 2.6079102691
friends 2.57349168896
okay 2.5431811901
thing 2.51064183145
ass 2.50489161177
bad 2.47795693509
hope 2.47577004242
school 2.47393609831
depression 2.44831675986
say 2.43705729885
yeah 2.42367599246
look 2.4174312008
